# contents

## 0.1 Import libraries

In [21]:
from pathlib import Path
from collections import Counter, defaultdict
import requests
import random

import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

import networkx as nx

print(f'numpy version: {np.__version__}')
print(f'pandas version: {pd.__version__}')
print(f'matplotlib version {mpl.__version__}')
print(f'networkx version {nx.__version__}')

numpy version: 1.19.0
pandas version: 1.0.5
matplotlib version 3.3.0
networkx version 2.5


## 0.2 function create_dir_save_file

In [22]:
def create_dir_save_file(dir_path: Path, url: str):
    """
    Check if the path exists and create it if it does not.
    Check if the file exists and download it if it does not.
    """
    if not dir_path.parents[0].exists():
        dir_path.parents[0].mkdir(parents=True)
        print(f'Directory Created: {dir_path.parents[0]}')
    else:
        print('Directory Exists')
        
    if not dir_path.exists():
        r = requests.get(url, allow_redirects=True)
        open(dir_path, 'wb').write(r.content)
        print(f'File Created: {dir_path.name}')
    else:
        print('File Exists')


In [15]:
;def


SyntaxError: invalid syntax (<ipython-input-15-e8ab097c9cd7>, line 1)